# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_ana/ana.470`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

In [10]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))


`qa_file_name_new` env not set. use the default `qa_file_name_new=prdf_reconstruction53877_qa.root`

In [11]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Openning QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

Openning QA file at `prdf_reconstruction53877_qa.root`

23014932684800

In [12]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}
// list inputs histograms if needed
 //qa_file_new ->ls();

In [13]:
%%cpp

int SiliconSeedsDrawTrackBasicInfo(TFile* qa_file_new)
{
    std::cout << "Silicon Seeds DrawTrackBasicInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return -1;
    }


    std::string histprefix = "h_SiliconSeedsQA_";
    // Get histograms

    TH1F *h_ntrack1d = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "nrecotracks1d").c_str()));
    TH2F *h_ntrack = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "nrecotracks").c_str()));
    TH2F *h_nmaps_nintt = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "nmaps_nintt").c_str()));
    TProfile2D *h_avgnclus_eta_phi = dynamic_cast<TProfile2D *>(qa_file_new->Get((histprefix + "avgnclus_eta_phi").c_str()));


    // Define canvas
    TCanvas *canvas = new TCanvas("trackbasic_info", "Track Basic Info", 1000, 1000);
    canvas->Divide(4, 1);  // 4 pads in one column

    // Pad 1: Draw h_ntrack1d
    canvas->cd(1);
    if (h_ntrack1d) {
        h_ntrack1d->SetTitle("Number of silicon-only tracks");
        h_ntrack1d->SetXTitle("Number of silicon-only tracks");
        h_ntrack1d->SetYTitle("Entries");
        h_ntrack1d->DrawCopy();
    } else {
        std::cerr << "Error: Missing histogram h_ntrack1d" << std::endl;
        return -1;
    }

    // Pad 2: Draw h_ntrack
    canvas->cd(2);
    if (h_ntrack) {
        h_ntrack->SetXTitle("#eta");
        h_ntrack->SetYTitle("#phi [rad]");
        h_ntrack->SetZTitle("Number of tracks");
        h_ntrack->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Error: Missing histogram h_ntrack" << std::endl;
        return -1;
    }

    // Pad 3: Draw h_nmaps_nintt
    canvas->cd(3);
    if (h_nmaps_nintt) {
        h_nmaps_nintt->SetXTitle("Number of MVTX clusters");
        h_nmaps_nintt->SetYTitle("Number of INTT clusters");
        h_nmaps_nintt->SetZTitle("Fraction");
        h_nmaps_nintt->SetMarkerSize(2.0);
        h_nmaps_nintt->Scale(1. / h_nmaps_nintt->Integral());
        h_nmaps_nintt->DrawCopy("colztext45");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Error: Missing histogram h_nmaps_nintt" << std::endl;
        return -1;
    }

    // Pad 4: Draw h_avgnclus_eta_phi
    canvas->cd(4);
    if (h_avgnclus_eta_phi) {
        h_avgnclus_eta_phi->SetXTitle("#eta");
        h_avgnclus_eta_phi->SetYTitle("#phi [rad]");
        h_avgnclus_eta_phi->SetZTitle("Average number of clusters per track");
        h_avgnclus_eta_phi->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Error: Missing histogram h_avgnclus_eta_phi" << std::endl;
        return -1;
    }

    // Add run information as text
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::ostringstream runnostream1;
    //runnostream1 << Name() << "_siliconseeds Track Info Run " << cl->RunNumber() << ", build " << cl->build();
    std::string runstring1 = runnostream1.str();
    PrintRun.DrawText(0.5, 0.95, runstring1.c_str());

    canvas->Update();
    std::cout << "DrawTrackBasicInfo Ending" << std::endl;
    return 0;
}


In [14]:
%%cpp
SiliconSeedsDrawTrackBasicInfo(qa_file_new);

Silicon Seeds DrawTrackBasicInfo() Beginning
DrawTrackBasicInfo Ending


In [15]:
%%cpp

void SiliconSeedsDrawTrackDCAInfo(TFile* qa_file_new) {
    std::cout << "Silicon Seeds DrawTrackDCAInfo() Beginning" << std::endl;
        
    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

     std::string histprefix = "h_SiliconSeedsQA_";

    // Define histograms
    TH2F *h_dcaxyorigin_phi = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "dcaxyorigin_phi").c_str()));
    TH2F *h_dcazorigin_phi = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "dcazorigin_phi").c_str()));
    TH2F *h_dcaxyvtx_phi = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "dcaxyvtx_phi").c_str()));
    TH2F *h_dcazvtx_phi = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "dcazvtx_phi").c_str()));

    // Create the canvas if it doesn't exist
    if (!gROOT->FindObject("trackdca_info")) {
        new TCanvas("trackdca_info", "Track DCA Info", 800, 600);
    }
    TCanvas *canvas = (TCanvas *)gROOT->FindObject("trackdca_info");
    canvas->Clear();  // Clear the canvas to avoid overwriting previous content

    // Divide canvas into 2x2 pads
    canvas->Divide(2, 2);

    // Plot dcaxyorigin_phi
    canvas->cd(1);
    if (h_dcaxyorigin_phi) {
        h_dcaxyorigin_phi->SetXTitle("#phi [rad]");
        h_dcaxyorigin_phi->SetYTitle("DCA_{xy} wrt origin [cm]");
        h_dcaxyorigin_phi->SetZTitle("Entries");
        h_dcaxyorigin_phi->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram dcaxyorigin_phi is missing" << std::endl;
    }

    // Plot dcaxyvtx_phi
    canvas->cd(2);
    if (h_dcaxyvtx_phi) {
        h_dcaxyvtx_phi->SetXTitle("#phi [rad]");
        h_dcaxyvtx_phi->SetYTitle("DCA_{xy} wrt vertex [cm]");
        h_dcaxyvtx_phi->SetZTitle("Entries");
        h_dcaxyvtx_phi->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram dcaxyvtx_phi is missing" << std::endl;
    }

    // Plot dcazorigin_phi
    canvas->cd(3);
    if (h_dcazorigin_phi) {
        h_dcazorigin_phi->SetXTitle("#phi [rad]");
        h_dcazorigin_phi->SetYTitle("DCA_{z} wrt origin [cm]");
        h_dcazorigin_phi->SetZTitle("Entries");
        h_dcazorigin_phi->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram dcazorigin_phi is missing" << std::endl;
    }

    // Plot dcazvtx_phi
    canvas->cd(4);
    if (h_dcazvtx_phi) {
        h_dcazvtx_phi->SetXTitle("#phi [rad]");
        h_dcazvtx_phi->SetYTitle("DCA_{z} wrt vertex [cm]");
        h_dcazvtx_phi->SetZTitle("Entries");
        h_dcazvtx_phi->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram dcazvtx_phi is missing" << std::endl;
    }

    // Add text to canvas
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();         // Set to normalized coordinates
    PrintRun.SetTextAlign(23); // Center/top alignment
    std::ostringstream runnostream1;
    std::string runstring1;
    //runnostream1 << "Silicon Seeds Track Info Run " << cl->RunNumber() << ", Build " << cl->build();
    runstring1 = runnostream1.str();

    // Draw text at the top of the canvas
    canvas->cd(0); 
    PrintRun.DrawText(0.5, 0.95, runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawTrackDCAInfo Ending" << std::endl;
}


In [16]:
%%cpp
SiliconSeedsDrawTrackDCAInfo(qa_file_new) 

Silicon Seeds DrawTrackDCAInfo() Beginning
DrawTrackDCAInfo Ending


In [17]:
%%cpp

void SiliconSeedsDrawTrackQualityInfo(TFile* qa_file_new) {
    std::cout << "Silicon Seeds DrawTrackQualityInfo() Beginning" << std::endl;

       // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

     std::string histprefix = "h_SiliconSeedsQA_";

    // Retrieve histograms
    TH1F *h_trackcrossing = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "trackcrossing").c_str()));
    TH1F *h_trackchi2ndf = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "trackchi2ndf").c_str()));
    TH1F *h_ntrack_isfromvtx = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "ntrack_isfromvtx").c_str()));

    // Create and define canvas
    if (!gROOT->FindObject("trackquality_info")) {
        new TCanvas("trackquality_info", "Track Quality Info", 800, 600);
    }
    TCanvas *canvas = (TCanvas *)gROOT->FindObject("trackquality_info");
    canvas->Clear();
    canvas->Divide(2, 2); // Divide into 2x2 pads for layout

    // Plot trackcrossing
    canvas->cd(1);
    if (h_trackcrossing) {
        h_trackcrossing->SetXTitle("Track crossing");
        h_trackcrossing->SetYTitle("Entries");
        h_trackcrossing->DrawCopy();
    } else {
        std::cerr << "Histogram trackcrossing is missing" << std::endl;
        return;
    }

    // Plot trackchi2ndf
    canvas->cd(2);
    if (h_trackchi2ndf) {
        gPad->SetLogy();
        h_trackchi2ndf->SetXTitle("Track #chi^{2}/ndof");
        h_trackchi2ndf->SetYTitle("Entries");
        h_trackchi2ndf->DrawCopy();
    } else {
        std::cerr << "Histogram trackchi2ndf is missing" << std::endl;
        return;
    }

    // Plot ntrack_isfromvtx
    canvas->cd(3);
    if (h_ntrack_isfromvtx) {
        h_ntrack_isfromvtx->SetTitle("Is track from a vertex");
        h_ntrack_isfromvtx->SetXTitle("Is track associated to a vertex");
        h_ntrack_isfromvtx->SetYTitle("Fraction");
        h_ntrack_isfromvtx->Scale(1. / h_ntrack_isfromvtx->Integral());
        h_ntrack_isfromvtx->GetYaxis()->SetRangeUser(0, 1.0);
        h_ntrack_isfromvtx->SetMarkerSize(2.5);
        h_ntrack_isfromvtx->GetXaxis()->SetNdivisions(3);
        h_ntrack_isfromvtx->DrawCopy("histtext0");
    } else {
        std::cerr << "Histogram ntrack_isfromvtx is missing" << std::endl;
        return;
    }

    // Add text information
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);

    std::ostringstream runnostream1;
    //runnostream1 << "Silicon Seeds Track Info Run " << cl->RunNumber() << ", Build " << cl->build();
    std::string runstring1 = runnostream1.str();

    canvas->cd(0);
    PrintRun.DrawText(0.5, 1.0, runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawTrackQualityInfo Ending" << std::endl;
}


In [18]:
%%cpp
SiliconSeedsDrawTrackQualityInfo(qa_file_new)

Silicon Seeds DrawTrackQualityInfo() Beginning
DrawTrackQualityInfo Ending


In [19]:
# Track Charge Information

In [20]:
%%cpp

void SiliconSeedsDrawTrackChargeInfo(TFile* qa_file_new) {
    std::cout << "Silicon Seeds DrawTrackChargeInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

     std::string histprefix = "h_SiliconSeedsQA_";

    // Retrieve histograms
    TH1F *h_trackpt_inclusive = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "trackpt").c_str()));
    TH1F *h_trackpt_pos = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "trackpt_pos").c_str()));
    TH1F *h_trackpt_neg = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "trackpt_neg").c_str()));
    TH1F *h_ntrack_IsPosCharge = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "ntrack_IsPosCharge").c_str()));

    // Create and define canvas
    if (!gROOT->FindObject("trackcharge_info")) {
        new TCanvas("trackcharge_info", "Track Charge Info", 800, 600);
    }
    TCanvas *canvas = (TCanvas *)gROOT->FindObject("trackcharge_info");
    canvas->Clear();
    canvas->Divide(2, 1); // Divide into 2 pads

    // Plot track pT distributions
    canvas->cd(1);
    if (h_trackpt_inclusive && h_trackpt_pos && h_trackpt_neg) {
        gPad->SetLogy();
        h_trackpt_inclusive->SetXTitle("Track p_{T} [GeV]");
        h_trackpt_inclusive->SetYTitle("Entries");
        h_trackpt_inclusive->SetLineColor(kBlack);
        h_trackpt_inclusive->DrawCopy();
        
        h_trackpt_pos->SetMarkerColor(kRed);
        h_trackpt_pos->SetLineColor(kRed);
        h_trackpt_pos->DrawCopy("same");
        
        h_trackpt_neg->SetMarkerColor(kBlue);
        h_trackpt_neg->SetLineColor(kBlue);
        h_trackpt_neg->DrawCopy("same");

        auto legend = new TLegend(0.45, 0.7, 0.8, 0.9);
        legend->AddEntry(h_trackpt_inclusive, "Inclusive", "pl");
        legend->AddEntry(h_trackpt_pos, "Positive charged", "pl");
        legend->AddEntry(h_trackpt_neg, "Negative charged", "pl");
        legend->Draw();
    } else {
        std::cerr << "One or more track pT histograms are missing" << std::endl;
        return;
    }

    // Plot positive charge fraction
    canvas->cd(2);
    if (h_ntrack_IsPosCharge) {
        h_ntrack_IsPosCharge->SetXTitle("Number of tracks with positive charge");
        h_ntrack_IsPosCharge->SetYTitle("Fraction");
        h_ntrack_IsPosCharge->Scale(1. / h_ntrack_IsPosCharge->Integral());
        h_ntrack_IsPosCharge->GetYaxis()->SetRangeUser(0, 1.0);
        h_ntrack_IsPosCharge->SetMarkerSize(2.5);
        h_ntrack_IsPosCharge->GetXaxis()->SetNdivisions(3);
        h_ntrack_IsPosCharge->DrawCopy("histtext0");
    } else {
        std::cerr << "Histogram ntrack_IsPosCharge is missing" << std::endl;
        return;
    }

    // Add text information
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);

    std::ostringstream runnostream1;
    //runnostream1 << "Silicon Seeds Track Charge Info Run " << qa_file_new->RunNumber() << ", Build " << qa_file_new->build();
    std::string runstring1 = runnostream1.str();

    canvas->cd();
    PrintRun.DrawText(0.5, 1.0, runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawTrackChargeInfo Ending" << std::endl;
}


In [21]:
%%cpp
SiliconSeedsDrawTrackChargeInfo(qa_file_new)

Silicon Seeds DrawTrackChargeInfo() Beginning
DrawTrackChargeInfo Ending


In [22]:
# Vertex Informaiton

In [25]:
%%cpp

void SiliconSeedsDrawVertexInfo(TFile* qa_file_new) {
    std::cout << "Silicon Seeds DrawVertexInfo() Beginning" << std::endl;

        // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

     std::string histprefix = "h_SiliconSeedsQA_";

    // Retrieve histograms
    TH1F *h_nvertex = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "nrecovertices").c_str()));
    TH2F *h_vx_vy = dynamic_cast<TH2F *>(qa_file_new->Get((histprefix + "vx_vy").c_str()));
    TH1F *h_vz = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "vz").c_str()));
    TH1F *h_vt = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "vt").c_str()));

    // Create and define canvas
    if (!gROOT->FindObject("vertex_info")) {
        new TCanvas("vertex_info", "Vertex Info", 1000, 800);
    }
    TCanvas *canvas = (TCanvas *)gROOT->FindObject("vertex_info");
    canvas->Clear();
    canvas->Divide(2, 2); // Divide into 4 pads

    // Plot number of vertices
    canvas->cd(1);
    if (h_nvertex) {
        gPad->SetLogy();
        h_nvertex->SetXTitle("Number of vertices");
        h_nvertex->SetYTitle("Entries");
        h_nvertex->DrawCopy();
    } else {
        std::cerr << "Histogram nrecovertices is missing" << std::endl;
        return;
    }

    // Plot vertex x vs y
    canvas->cd(2);
    if (h_vx_vy) {
        h_vx_vy->SetXTitle("Vertex x [cm]");
        h_vx_vy->SetYTitle("Vertex y [cm]");
        h_vx_vy->SetZTitle("Entries");
        h_vx_vy->DrawCopy("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram vx_vy is missing" << std::endl;
        return;
    }

    // Plot vertex z
    canvas->cd(3);
    if (h_vz) {
        h_vz->SetXTitle("Vertex z [cm]");
        h_vz->SetYTitle("Entries");
        h_vz->DrawCopy();
    } else {
        std::cerr << "Histogram vz is missing" << std::endl;
        return;
    }

    // Plot vertex time
    canvas->cd(4);
    if (h_vt) {
        h_vt->SetXTitle("Vertex t [ns]");
        h_vt->SetYTitle("Entries");
        h_vt->DrawCopy("colz");
    } else {
        std::cerr << "Histogram vt is missing" << std::endl;
        return;
    }

    // Add text information
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);

    std::ostringstream runnostream1;
    //runnostream1 << "Silicon Seeds Vertex Info Run " << qa_file_new->RunNumber() << ", Build " << qa_file_new->build();
    std::string runstring1 = runnostream1.str();

    canvas->cd();
    PrintRun.DrawText(0.5, 1.0, runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawVertexInfo Ending" << std::endl;
}


In [26]:
%%cpp
SiliconSeedsDrawVertexInfo(qa_file_new)

Silicon Seeds DrawVertexInfo() Beginning
DrawVertexInfo Ending


In [ ]:
# Vertex Quality Information

In [27]:
%%cpp

void SiliconSeedsDrawVertexQualityInfo(TFile* qa_file_new) {
    std::cout << "Silicon Seeds DrawVertexQualityInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

     std::string histprefix = "h_SiliconSeedsQA_";

    // Retrieve histograms
    TH1F *h_vcrossing = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "vertexcrossing").c_str()));
    TH1F *h_vchi2dof = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "vertexchi2dof").c_str()));
    TH1F *h_ntrackpervertex = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "ntrackspervertex").c_str()));

    // Create and define canvas
    if (!gROOT->FindObject("vertexquality_info")) {
        new TCanvas("vertexquality_info", "Vertex Quality Info", 1000, 800);
    }
    TCanvas *canvas = (TCanvas *)gROOT->FindObject("vertexquality_info");
    canvas->Clear();
    canvas->Divide(2, 2); // Divide into 3 pads (one remains empty)

    // Plot vertex crossing
    canvas->cd(1);
    if (h_vcrossing) {
        h_vcrossing->SetXTitle("Vertex crossing");
        h_vcrossing->SetYTitle("Entries");
        h_vcrossing->DrawCopy();
    } else {
        std::cerr << "Histogram vertexcrossing is missing" << std::endl;
        return;
    }

    // Plot vertex chi2/ndof
    canvas->cd(2);
    if (h_vchi2dof) {
        h_vchi2dof->SetXTitle("Vertex #chi^{2}/ndof");
        h_vchi2dof->SetYTitle("Entries");
        h_vchi2dof->DrawCopy();
    } else {
        std::cerr << "Histogram vertexchi2dof is missing" << std::endl;
        return;
    }

    // Plot number of tracks per vertex
    canvas->cd(3);
    if (h_ntrackpervertex) {
        gPad->SetLogy();
        h_ntrackpervertex->SetXTitle("Number of tracks per vertex");
        h_ntrackpervertex->SetYTitle("Entries");
        h_ntrackpervertex->DrawCopy();
    } else {
        std::cerr << "Histogram ntrackspervertex is missing" << std::endl;
        return;
    }

    // Add text information
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);

    std::ostringstream runnostream1;
    //runnostream1 << "Silicon Seeds Vertex Quality Info Run " << cl->RunNumber() << ", Build " << cl->build();
    std::string runstring1 = runnostream1.str();

    canvas->cd(4);
    PrintRun.DrawText(0.5, 1.0, runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawVertexQualityInfo Ending" << std::endl;
}


In [28]:
%%cpp
SiliconSeedsDrawVertexQualityInfo(qa_file_new)

Silicon Seeds DrawVertexQualityInfo() Beginning
DrawVertexQualityInfo Ending
